In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
# Add the fourier_learning_ibm package to the path
# This is necessary only when running the notebook in the docker container
import sys, pprint

sys.path.append("/home/jovyan/fourier_learning_ibm/")
pprint.pprint(sys.path)

# This is necessary only when running the notebook in the docker container
%cd fourier_learning_ibm/

['/home/jovyan',
 '/opt/conda/lib/python311.zip',
 '/opt/conda/lib/python3.11',
 '/opt/conda/lib/python3.11/lib-dynload',
 '',
 '/opt/conda/lib/python3.11/site-packages',
 '/home/jovyan/fourier_learning_ibm/']
/home/jovyan/fourier_learning_ibm


In [5]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
import pandas as pd
import os
from heisenberg import (
    HeisenbergModel,
    get_graph,
    get_positions,
    get_initial_layout,
    get_prob0,
    # extract_probs,
)
from qiskit import transpile
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler, Batch
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel
import pickle
import networkx as nx
from datetime import datetime, timezone
import json
import math

In [6]:
import warnings

warnings.filterwarnings("ignore")

### Backend の設定

In [7]:
# Option1: Use IBM Quantum backend.

# Set up the Qiskit Runtime service (this is a one-time setup)
# QiskitRuntimeService.save_account(
#     token="YOUR_API_TOKEN",
#     channel="ibm_quantum",
# )

# Load saved credentials
service = QiskitRuntimeService()
# backend_qpu = service.least_busy(simulator=False, interactional=True)
backend_qpu = service.backend("ibm_marrakesh")
print(f"Using backend QPU: {backend_qpu}")

Using backend QPU: <IBMBackend('ibm_marrakesh')>


In [8]:
# Option2: Use local AerSimulator as the backend.

# Noiseless simulator
backend_sim_noiseless = AerSimulator()
print(f"Using backend noiseless simulator: {backend_sim_noiseless}")
print()

# Noise model
noise_backend = NoiseModel.from_backend(backend_qpu)
print(noise_backend)
print()

# Noisy simulator
backend_sim_noisy = AerSimulator(noise_model=noise_backend)
print(f"Using backend noisy simulator: {backend_sim_noisy}")

Using backend noiseless simulator: AerSimulator('aer_simulator')

NoiseModel:
  Basis gates: ['cz', 'delay', 'for_loop', 'id', 'if_else', 'measure', 'reset', 'rz', 'switch_case', 'sx', 'x']
  Instructions with noise: ['sx', 'cz', 'reset', 'id', 'measure', 'x']
  Qubits with noise: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155]
  Specific qubit errors: [('sx', (0,)), ('sx', 

In [9]:
n_qubits = 4

# Load parameters
with open(f"results/get_param/{n_qubits}Q_param.json", "r") as f:
    param = json.load(f)
C = param["C"]
K = param["K"]
n_features = (
    K + 1
)  # 実数部分の特徴量の数。1 は t=0, つまり定数項。虚数も含んだ特徴量の数は 2K+1

print(f"C: {C}")
print(f"K: {K}")

# Load dataset (orient="records")
df_n_step = pd.read_json(
    f"results/get_param/{n_qubits}Q_optimal_n_step.json", orient="records"
)
times = df_n_step["t"].values
print(f"times: {times}")
display(df_n_step)

# Load dataset (orient="records")
df_dataset = pd.read_json(
    f"results/get_dataset/{n_qubits}Q_dataset.json", orient="records"
)
all_Js = df_dataset["Js"].values
all_expected_values = df_dataset["expected_value"].values
n_samples = len(all_Js)
display(df_dataset)

C: 9
K: 8
times: [0.         0.34906585 0.6981317  1.04719755 1.3962634  1.74532925
 2.0943951  2.44346095 2.7925268 ]


,t,n_step,sta_fidelity
0,0.000000,1,1.000000
1,0.349066,1,0.974201
2,0.698132,2,0.950947
3,1.047198,2,0.946502
4,1.396263,3,0.923480
5,1.745329,3,0.911794
6,2.094395,4,0.892342
7,2.443461,5,0.876343
8,2.792527,5,0.860245


,Js,expected_value
0,"[0.5479120971, -0.12224312050000001, 0.7171958...",13.899220
1,"[0.3947360581, -0.8116453042, 0.9512447033]",28.585213
2,"[0.5222794040000001, 0.5721286106, -0.7437727346]",2.653082
3,"[-0.0992281242, -0.2584039515, 0.8535299777]",7.222891
4,"[0.2877302402, 0.6455232265, -0.11317160230000...",0.865078
5,"[-0.5455225564, 0.109169574, -0.8723654878]",1.138950
6,"[0.655262344, 0.2633287982, 0.5161754802]",9.060659
7,"[-0.29094806370000004, 0.9413960488, 0.7862422...",4.367027
8,"[0.5567669941, -0.6107225843, -0.0665579925]",4.566077
9,"[-0.9123924684, -0.6914210159, 0.3660979065]",3.720380


# Create Fourier feature

## Trotter simulation (Exact)

In [10]:
data = []
probs_phase0_exact = {}
probs_phase1_exact = {}
probs_phase2_exact = {}
probs_phase3_exact = {}

# Generate the dataset with Fourier features
for i in range(n_samples):
    print(f"Calculating features for sample {i}/{n_samples}")
    Js = all_Js[i]
    G = get_graph(n_qubits, Js)
    heisenberg = HeisenbergModel(n_qubits, G)

    lambda_ref = np.sum(Js)  # Reference eigenvalue.
    features_exact = []

    probs_phase0_exact[f"sample{i}"] = {}
    probs_phase1_exact[f"sample{i}"] = {}
    probs_phase2_exact[f"sample{i}"] = {}
    probs_phase3_exact[f"sample{i}"] = {}

    # Compute the Fourier features for different times
    for k in range(n_features):
        final_state_phase0, _ = heisenberg.exact_simulation(times[k], phase=0)
        final_state_phase1, _ = heisenberg.exact_simulation(times[k], phase=1)
        final_state_phase2, _ = heisenberg.exact_simulation(times[k], phase=2)
        final_state_phase3, _ = heisenberg.exact_simulation(times[k], phase=3)

        # "00...00" がなければ確率 0 を取得
        prob_phase0 = final_state_phase0.probabilities_dict().get("0" * n_qubits, 0)
        prob_phase1 = final_state_phase1.probabilities_dict().get("0" * n_qubits, 0)
        prob_phase2 = final_state_phase2.probabilities_dict().get("0" * n_qubits, 0)
        prob_phase3 = final_state_phase3.probabilities_dict().get("0" * n_qubits, 0)

        probs_phase0_exact[f"sample{i}"][f"f_{k}"] = prob_phase0
        probs_phase1_exact[f"sample{i}"][f"f_{k}"] = prob_phase1
        probs_phase2_exact[f"sample{i}"][f"f_{k}"] = prob_phase2
        probs_phase3_exact[f"sample{i}"][f"f_{k}"] = prob_phase3

        inner_product = np.exp(-1j * lambda_ref * times[k]) * (
            (prob_phase0 - prob_phase1) + 1j * (prob_phase2 - prob_phase3)
        )

        features_exact.append(inner_product.real)
        if k != 0:
            features_exact.append(inner_product.imag)
    data.append([i, *features_exact, all_expected_values[i]])

# Create column names for the DataFrame
columns = []
columns.append("sample_id")
for k in range(n_features):
    columns.append(f"f_{k} Re")
    if k != 0:
        columns.append(f"f_{k} Im")
columns.append("expected_value")

# Convert to a DataFrame
df_exact = pd.DataFrame(data, columns=columns)
display(df_exact)

# Save the exact dataset
path = "results/fourier_feature_sim"
df_exact.to_json(f"{path}/result_exact.json", orient="records", indent=4)
with open(f"{path}/probs_phase0_exact.json", "w") as f:
    json.dump(probs_phase0_exact, f)

with open(f"{path}/probs_phase1_exact.json", "w") as f:
    json.dump(probs_phase1_exact, f)

with open(f"{path}/probs_phase2_exact.json", "w") as f:
    json.dump(probs_phase2_exact, f)

with open(f"{path}/probs_phase3_exact.json", "w") as f:
    json.dump(probs_phase3_exact, f)

Calculating features for sample 0/55
Calculating features for sample 1/55
Calculating features for sample 2/55
Calculating features for sample 3/55
Calculating features for sample 4/55
Calculating features for sample 5/55
Calculating features for sample 6/55
Calculating features for sample 7/55
Calculating features for sample 8/55
Calculating features for sample 9/55
Calculating features for sample 10/55
Calculating features for sample 11/55
Calculating features for sample 12/55
Calculating features for sample 13/55
Calculating features for sample 14/55
Calculating features for sample 15/55
Calculating features for sample 16/55
Calculating features for sample 17/55
Calculating features for sample 18/55
Calculating features for sample 19/55
Calculating features for sample 20/55
Calculating features for sample 21/55
Calculating features for sample 22/55
Calculating features for sample 23/55
Calculating features for sample 24/55
Calculating features for sample 25/55
Calculating features f

,sample_id,f_0 Re,f_1 Re,f_1 Im,f_2 Re,f_2 Im,f_3 Re,f_3 Im,f_4 Re,f_4 Im,f_5 Re,f_5 Im,f_6 Re,f_6 Im,f_7 Re,f_7 Im,f_8 Re,f_8 Im,expected_value
0,0,1.0,0.718041,0.383606,0.195441,0.340886,-0.061805,0.044172,-0.059755,-0.029632,-0.207918,0.171675,-0.599948,0.220433,-0.802551,-0.096272,-0.490669,-0.402475,13.899220
1,1,1.0,0.533219,0.556411,-0.176896,0.330914,-0.190092,-0.225399,0.255385,-0.298390,0.379184,-0.034141,0.197245,-0.017423,0.179823,-0.100772,0.162744,0.154607,28.585213
2,2,1.0,0.778346,-0.246211,0.301691,-0.343953,-0.040665,-0.286528,-0.018982,-0.205568,0.232292,-0.226135,0.357286,-0.330143,0.130089,-0.371267,-0.320885,-0.229954,2.653082
3,3,1.0,0.771926,0.295819,0.242453,0.300258,-0.238066,-0.049999,-0.378689,-0.537668,-0.151334,-0.827320,0.207169,-0.716856,0.387829,-0.285992,0.243577,0.167489,7.222891
4,4,1.0,0.963873,-0.163293,0.863299,-0.319218,0.719016,-0.457295,0.557826,-0.563322,0.404492,-0.622343,0.275721,-0.624241,0.178362,-0.569336,0.111796,-0.471004,0.865078
5,5,1.0,0.652979,-0.392155,0.091830,-0.237883,-0.061567,0.099905,-0.026271,0.051280,-0.279651,-0.158154,-0.556160,-0.021664,-0.312819,0.214764,0.192527,-0.045099,1.138950
6,6,1.0,0.801611,0.252730,0.416358,0.227315,0.199764,0.024042,0.200110,-0.014849,0.134899,0.227875,-0.180995,0.480511,-0.546562,0.444028,-0.624683,0.188821,9.060659
7,7,1.0,0.831500,-0.164043,0.476650,-0.340292,0.220555,-0.422985,0.190423,-0.288624,0.285566,0.003609,0.367036,0.211216,0.419182,0.156153,0.486711,-0.076606,4.367027
8,8,1.0,0.855184,0.356904,0.484418,0.563724,0.050315,0.535060,-0.258042,0.286645,-0.308841,-0.072210,-0.084979,-0.386555,0.310219,-0.522861,0.700787,-0.425811,4.566077
9,9,1.0,0.783503,0.073778,0.339573,0.228951,0.056574,0.356156,0.097964,0.268420,0.292518,-0.018056,0.402635,-0.237244,0.390137,-0.168979,0.364223,0.084275,3.720380


## Trotter simulation (Simulator, Noise free)

In [11]:
# Create circuits
# 1 batch - 1 sample
circuits_phase0 = {}
circuits_phase1 = {}
circuits_phase2 = {}
circuits_phase3 = {}
exec_circuits_phase0 = {}
exec_circuits_phase1 = {}
exec_circuits_phase2 = {}
exec_circuits_phase3 = {}
lambda_refs = {}

for i in range(n_samples):
    print(f"Calculating features for sample {i}/{n_samples}")
    Js = all_Js[i]
    G = get_graph(n_qubits, Js)
    heisenberg = HeisenbergModel(n_qubits, G)

    # lambda_ref はこの後も使うので、辞書として保存
    lambda_refs[f"sample{i}"] = np.sum(Js)  # Reference eigenvalue.

    circuits_phase0[f"sample{i}"] = {}
    circuits_phase1[f"sample{i}"] = {}
    circuits_phase2[f"sample{i}"] = {}
    circuits_phase3[f"sample{i}"] = {}
    exec_circuits_phase0[f"sample{i}"] = {}
    exec_circuits_phase1[f"sample{i}"] = {}
    exec_circuits_phase2[f"sample{i}"] = {}
    exec_circuits_phase3[f"sample{i}"] = {}
    for k in range(n_features):
        # n_steps = get_n_steps(times[k])
        n_step = df_n_step[df_n_step["t"] == times[k]]["n_step"].values[0]

        circuit_phase0 = heisenberg.get_circuit(times[k], n_step, phase=0)
        circuit_phase1 = heisenberg.get_circuit(times[k], n_step, phase=1)
        circuit_phase2 = heisenberg.get_circuit(times[k], n_step, phase=2)
        circuit_phase3 = heisenberg.get_circuit(times[k], n_step, phase=3)

        exec_circuit_phase0 = transpile(circuit_phase0, backend_qpu)
        exec_circuit_phase1 = transpile(circuit_phase1, backend_qpu)
        exec_circuit_phase2 = transpile(circuit_phase2, backend_qpu)
        exec_circuit_phase3 = transpile(circuit_phase3, backend_qpu)

        circuits_phase0[f"sample{i}"][f"f_{k}"] = circuit_phase0
        circuits_phase1[f"sample{i}"][f"f_{k}"] = circuit_phase1
        circuits_phase2[f"sample{i}"][f"f_{k}"] = circuit_phase2
        circuits_phase3[f"sample{i}"][f"f_{k}"] = circuit_phase3
        exec_circuits_phase0[f"sample{i}"][f"f_{k}"] = exec_circuit_phase0
        exec_circuits_phase1[f"sample{i}"][f"f_{k}"] = exec_circuit_phase1
        exec_circuits_phase2[f"sample{i}"][f"f_{k}"] = exec_circuit_phase2
        exec_circuits_phase3[f"sample{i}"][f"f_{k}"] = exec_circuit_phase3

Calculating features for sample 0/55
Calculating features for sample 1/55
Calculating features for sample 2/55
Calculating features for sample 3/55
Calculating features for sample 4/55
Calculating features for sample 5/55
Calculating features for sample 6/55
Calculating features for sample 7/55
Calculating features for sample 8/55
Calculating features for sample 9/55
Calculating features for sample 10/55
Calculating features for sample 11/55
Calculating features for sample 12/55
Calculating features for sample 13/55
Calculating features for sample 14/55
Calculating features for sample 15/55
Calculating features for sample 16/55
Calculating features for sample 17/55
Calculating features for sample 18/55
Calculating features for sample 19/55
Calculating features for sample 20/55
Calculating features for sample 21/55
Calculating features for sample 22/55
Calculating features for sample 23/55
Calculating features for sample 24/55
Calculating features for sample 25/55
Calculating features f

In [12]:
# Run jobs in batch

# For AerSimulator, we can't use job ids. Instead, we store the jobs in a list.
jobs = []
with Batch(backend=backend_sim_noiseless):
    sampler = Sampler()

    for i in range(n_samples):
        print(f"Running circuits for sample {i}/{n_samples}")
        exec_circuits_per_sample = []
        exec_circuits_per_sample += [
            exec_circuits_phase0[f"sample{i}"][f"f_{k}"] for k in range(n_features)
        ]
        exec_circuits_per_sample += [
            exec_circuits_phase1[f"sample{i}"][f"f_{k}"] for k in range(n_features)
        ]
        exec_circuits_per_sample += [
            exec_circuits_phase2[f"sample{i}"][f"f_{k}"] for k in range(n_features)
        ]
        exec_circuits_per_sample += [
            exec_circuits_phase3[f"sample{i}"][f"f_{k}"] for k in range(n_features)
        ]

        job = sampler.run(exec_circuits_per_sample)
        jobs.append(job)

Running circuits for sample 0/55
Running circuits for sample 1/55
Running circuits for sample 2/55
Running circuits for sample 3/55
Running circuits for sample 4/55
Running circuits for sample 5/55
Running circuits for sample 6/55
Running circuits for sample 7/55
Running circuits for sample 8/55
Running circuits for sample 9/55
Running circuits for sample 10/55
Running circuits for sample 11/55
Running circuits for sample 12/55
Running circuits for sample 13/55
Running circuits for sample 14/55
Running circuits for sample 15/55
Running circuits for sample 16/55
Running circuits for sample 17/55
Running circuits for sample 18/55
Running circuits for sample 19/55
Running circuits for sample 20/55
Running circuits for sample 21/55
Running circuits for sample 22/55
Running circuits for sample 23/55
Running circuits for sample 24/55
Running circuits for sample 25/55
Running circuits for sample 26/55
Running circuits for sample 27/55
Running circuits for sample 28/55
Running circuits for sam

In [14]:
check_list = []
for i in range(n_samples):
    job_id = jobs[i].job_id()
    check_list.append(
        {
            "Job ID": job_id,
            "sample_id": i,
            "Status": job.status().name,
        }
    )

check_df = pd.DataFrame(check_list)
display(check_df)

,Job ID,sample_id,Status
0,a97b8d40-3e4c-4634-8a7d-f29536e6ba84,0,DONE
1,3493d128-f1b0-4b70-b5fd-9bc12b230bb0,1,DONE
2,e70492db-f236-40a2-b4d3-5fb1018c6c75,2,DONE
3,d44d1f68-6114-49ff-b651-1e7050997781,3,DONE
4,cc9495ae-eb3d-4d97-bda9-5ca3826041a5,4,DONE
5,915897c7-52aa-432d-9ef2-6697dcdc7754,5,DONE
6,939527ee-3823-413c-a6b3-c548412b0553,6,DONE
7,6dc07a51-e3f0-4290-8368-521b9f92d3f6,7,DONE
8,ef452c16-8d1f-4519-8de1-687f9b324347,8,DONE
9,5acf1a32-bf98-47d8-a425-e054bee39e87,9,DONE


In [15]:
# Post-process
data_sim = []
probs_phase0_sim = {}
probs_phase1_sim = {}
probs_phase2_sim = {}
probs_phase3_sim = {}

for i in range(n_samples):
    features = []
    probs_phase0_sim[f"sample{i}"] = {}
    probs_phase1_sim[f"sample{i}"] = {}
    probs_phase2_sim[f"sample{i}"] = {}
    probs_phase3_sim[f"sample{i}"] = {}

    # Compute the Fourier features for different times
    for k in range(n_features):
        # Get results of each phase in a batch
        results_phase0 = jobs[i].result()[:n_features]
        results_phase1 = jobs[i].result()[n_features : 2 * n_features]
        results_phase2 = jobs[i].result()[2 * n_features : 3 * n_features]
        results_phase3 = jobs[i].result()[3 * n_features :]

        prob_phase0 = get_prob0(results_phase0[k], n_qubits)
        prob_phase1 = get_prob0(results_phase1[k], n_qubits)
        prob_phase2 = get_prob0(results_phase2[k], n_qubits)
        prob_phase3 = get_prob0(results_phase3[k], n_qubits)

        probs_phase0_sim[f"sample{i}"][f"f_{k}"] = prob_phase0
        probs_phase1_sim[f"sample{i}"][f"f_{k}"] = prob_phase1
        probs_phase2_sim[f"sample{i}"][f"f_{k}"] = prob_phase2
        probs_phase3_sim[f"sample{i}"][f"f_{k}"] = prob_phase3

        inner_product = np.exp(-1j * lambda_refs[f"sample{i}"] * times[k]) * (
            (prob_phase0 - prob_phase1) + 1j * (prob_phase2 - prob_phase3)
        )

        features.append(inner_product.real)
        if k != 0:
            features.append(inner_product.imag)
    data_sim.append([i, *features, all_expected_values[i]])

# Create column names for the DataFrame
columns = []
columns.append("sample_id")
for k in range(n_features):
    columns.append(f"f_{k} Re")
    if k != 0:
        columns.append(f"f_{k} Im")
columns.append("expected_value")

# Convert to a DataFrame
df_sim = pd.DataFrame(data_sim, columns=columns)
display(df_sim)

# Save the simulation data
path = "results/fourier_feature_sim"
df_sim.to_json(f"{path}/data_sim_df.json", orient="records", indent=4)
with open(f"{path}/probs_phase0_sim.json", "w") as f:
    json.dump(probs_phase0_sim, f)

with open(f"{path}/probs_phase1_sim.json", "w") as f:
    json.dump(probs_phase1_sim, f)

with open(f"{path}/probs_phase2_sim.json", "w") as f:
    json.dump(probs_phase2_sim, f)

with open(f"{path}/probs_phase3_sim.json", "w") as f:
    json.dump(probs_phase3_sim, f)

,sample_id,f_0 Re,f_1 Re,f_1 Im,f_2 Re,f_2 Im,f_3 Re,f_3 Im,f_4 Re,f_4 Im,f_5 Re,f_5 Im,f_6 Re,f_6 Im,f_7 Re,f_7 Im,f_8 Re,f_8 Im,expected_value
0,0,1.0,0.685223,0.373516,0.202872,0.319284,-0.098598,0.064114,-0.080857,-0.030219,-0.198207,0.159077,-0.616160,0.227606,-0.794261,-0.090578,-0.474915,-0.405533,13.899220
1,1,1.0,0.565818,0.507391,-0.185268,0.330483,-0.288532,-0.144232,0.141545,-0.313013,0.462913,-0.107727,0.406158,-0.002674,0.198744,-0.046210,0.082487,-0.052274,28.585213
2,2,1.0,0.774860,-0.187804,0.288307,-0.312589,-0.105804,-0.307725,-0.067350,-0.238537,0.124766,-0.288007,0.290323,-0.391044,0.211238,-0.357095,-0.152283,-0.205859,2.653082
3,3,1.0,0.761695,0.307980,0.232180,0.298288,-0.227853,-0.077849,-0.397569,-0.555839,-0.209228,-0.802733,0.157037,-0.699704,0.302187,-0.304851,0.228533,0.137403,7.222891
4,4,1.0,0.960219,-0.134011,0.854612,-0.348670,0.714605,-0.451360,0.551436,-0.556292,0.424153,-0.618915,0.272597,-0.600031,0.196092,-0.539294,0.139504,-0.476582,0.865078
5,5,1.0,0.677002,-0.376721,0.050267,-0.238598,-0.068364,0.134455,-0.025923,0.029546,-0.309609,-0.136198,-0.547793,-0.001315,-0.327563,0.224139,0.200974,-0.076207,1.138950
6,6,1.0,0.804954,0.268552,0.420023,0.222340,0.162439,0.015029,0.173583,-0.017752,0.142232,0.232933,-0.163298,0.513797,-0.555703,0.466847,-0.603102,0.194295,9.060659
7,7,1.0,0.816333,-0.105649,0.465833,-0.314453,0.147818,-0.354267,0.159081,-0.308737,0.297052,-0.003380,0.385571,0.307582,0.357215,0.317345,0.367317,0.162751,4.367027
8,8,1.0,0.864021,0.325687,0.495525,0.562185,0.066285,0.509465,-0.261257,0.272596,-0.323630,-0.079079,-0.104638,-0.294244,0.293240,-0.495804,0.673965,-0.400495,4.566077
9,9,1.0,0.791677,0.025221,0.312656,0.181199,-0.026251,0.320907,0.035513,0.293371,0.299741,0.062993,0.391646,-0.293574,0.366669,-0.326398,0.245428,-0.100848,3.720380


## Trotter simulation (Simulator, Noisy)

In [ ]:
# ToDo below

In [12]:
# Create circuits
# 1 batch - 1 sample
circuits_phase0 = {}
circuits_phase1 = {}
circuits_phase2 = {}
circuits_phase3 = {}
exec_circuits_phase0 = {}
exec_circuits_phase1 = {}
exec_circuits_phase2 = {}
exec_circuits_phase3 = {}

for i in range(n_samples):
    print(f"Creating circuits for sample {i}/{n_samples}")
    Js = all_Js[i]
    heisenberg_sim_noisy = HeisenbergModel(
        n_qubits, graphs[i], backend=backend_sim_noisy
    )

    circuits_phase0[f"sample{i}"] = {}
    circuits_phase1[f"sample{i}"] = {}
    circuits_phase2[f"sample{i}"] = {}
    circuits_phase3[f"sample{i}"] = {}
    exec_circuits_phase0[f"sample{i}"] = {}
    exec_circuits_phase1[f"sample{i}"] = {}
    exec_circuits_phase2[f"sample{i}"] = {}
    exec_circuits_phase3[f"sample{i}"] = {}
    for k in range(n_features):
        n_steps = get_n_steps(times[k])
        circuit_phase0, exec_circuit_phase0 = (
            heisenberg_sim_noisy.get_trotter_simulation_circuit(
                times[k], n_steps, phase=0
            )
        )
        circuit_phase1, exec_circuit_phase1 = (
            heisenberg_sim_noisy.get_trotter_simulation_circuit(
                times[k], n_steps, phase=1
            )
        )
        circuit_phase2, exec_circuit_phase2 = (
            heisenberg_sim_noisy.get_trotter_simulation_circuit(
                times[k], n_steps, phase=2
            )
        )
        circuit_phase3, exec_circuit_phase3 = (
            heisenberg_sim_noisy.get_trotter_simulation_circuit(
                times[k], n_steps, phase=3
            )
        )

        circuits_phase0[f"sample{i}"][f"f_{k}"] = circuit_phase0
        circuits_phase1[f"sample{i}"][f"f_{k}"] = circuit_phase1
        circuits_phase2[f"sample{i}"][f"f_{k}"] = circuit_phase2
        circuits_phase3[f"sample{i}"][f"f_{k}"] = circuit_phase3
        exec_circuits_phase0[f"sample{i}"][f"f_{k}"] = exec_circuit_phase0
        exec_circuits_phase1[f"sample{i}"][f"f_{k}"] = exec_circuit_phase1
        exec_circuits_phase2[f"sample{i}"][f"f_{k}"] = exec_circuit_phase2
        exec_circuits_phase3[f"sample{i}"][f"f_{k}"] = exec_circuit_phase3

Creating circuits for sample 0/56
Creating circuits for sample 1/56
Creating circuits for sample 2/56
Creating circuits for sample 3/56
Creating circuits for sample 4/56
Creating circuits for sample 5/56
Creating circuits for sample 6/56
Creating circuits for sample 7/56
Creating circuits for sample 8/56
Creating circuits for sample 9/56
Creating circuits for sample 10/56
Creating circuits for sample 11/56
Creating circuits for sample 12/56
Creating circuits for sample 13/56
Creating circuits for sample 14/56
Creating circuits for sample 15/56
Creating circuits for sample 16/56
Creating circuits for sample 17/56
Creating circuits for sample 18/56
Creating circuits for sample 19/56
Creating circuits for sample 20/56
Creating circuits for sample 21/56
Creating circuits for sample 22/56
Creating circuits for sample 23/56
Creating circuits for sample 24/56
Creating circuits for sample 25/56
Creating circuits for sample 26/56
Creating circuits for sample 27/56
Creating circuits for sample 2

In [13]:
# Run jobs in batch
jobs = []  # For AerSimulator, we can't use job ids.
with Batch(backend=backend_sim_noisy):
    sampler = Sampler()

    for i in range(n_samples):
        print(f"Running circuits for sample {i}/{n_samples}")
        exec_circuits_per_sample = []
        exec_circuits_per_sample += [
            exec_circuits_phase0[f"sample{i}"][f"f_{k}"] for k in range(n_features)
        ]
        exec_circuits_per_sample += [
            exec_circuits_phase1[f"sample{i}"][f"f_{k}"] for k in range(n_features)
        ]
        exec_circuits_per_sample += [
            exec_circuits_phase2[f"sample{i}"][f"f_{k}"] for k in range(n_features)
        ]
        exec_circuits_per_sample += [
            exec_circuits_phase3[f"sample{i}"][f"f_{k}"] for k in range(n_features)
        ]

        job = sampler.run(exec_circuits_per_sample)
        jobs.append(job)

Running circuits for sample 0/56
Running circuits for sample 1/56
Running circuits for sample 2/56
Running circuits for sample 3/56
Running circuits for sample 4/56
Running circuits for sample 5/56
Running circuits for sample 6/56
Running circuits for sample 7/56
Running circuits for sample 8/56
Running circuits for sample 9/56
Running circuits for sample 10/56
Running circuits for sample 11/56
Running circuits for sample 12/56
Running circuits for sample 13/56
Running circuits for sample 14/56
Running circuits for sample 15/56
Running circuits for sample 16/56
Running circuits for sample 17/56
Running circuits for sample 18/56
Running circuits for sample 19/56
Running circuits for sample 20/56
Running circuits for sample 21/56
Running circuits for sample 22/56
Running circuits for sample 23/56
Running circuits for sample 24/56
Running circuits for sample 25/56
Running circuits for sample 26/56
Running circuits for sample 27/56
Running circuits for sample 28/56
Running circuits for sam

In [14]:
check_list = []
for i in range(n_samples):
    job_id = jobs[i].job_id()
    check_list.append(
        {
            "Job ID": job_id,
            "sample_id": i,
            "Status": job.status().name,
        }
    )

check_df = pd.DataFrame(check_list)
display(check_df)

,Job ID,sample_id,Status
0,dda4cbd8-1907-4f86-a525-21c00ee5a8a9,0,DONE
1,b5a47fdb-9b1e-4449-a1f7-a74c36247c46,1,DONE
2,ee20d17a-f401-4795-aa3a-3d77ce2af662,2,DONE
3,ab73285d-4fbb-42fe-bb40-dfae906bf1ec,3,DONE
4,954fe2c3-2bfb-4bce-adf7-c511531f2761,4,DONE
5,144111b7-e871-451a-a461-5678299761c8,5,DONE
6,2a6b1446-aa41-4b0f-b563-7dfe29f1af0c,6,DONE
7,a97fc745-6918-4657-b3ce-4958aa251b0a,7,DONE
8,f8eb632f-686f-4767-9b66-7f3c43fbe6b2,8,DONE
9,f1721ed8-f854-48dd-aa04-4b5f925e0fee,9,DONE


In [15]:
# Post-process
data_sim_noisy = []
probs_phase0_sim_noisy = {}
probs_phase1_sim_noisy = {}
probs_phase2_sim_noisy = {}
probs_phase3_sim_noisy = {}

for i in range(n_samples):
    features = []
    probs_phase0_sim_noisy[f"sample{i}"] = {}
    probs_phase1_sim_noisy[f"sample{i}"] = {}
    probs_phase2_sim_noisy[f"sample{i}"] = {}
    probs_phase3_sim_noisy[f"sample{i}"] = {}

    # Compute the Fourier features for different times
    for k in range(n_features):
        # Get results of each phase in a batch
        results_phase0 = jobs[i].result()[:n_features]
        results_phase1 = jobs[i].result()[n_features : 2 * n_features]
        results_phase2 = jobs[i].result()[2 * n_features : 3 * n_features]
        results_phase3 = jobs[i].result()[3 * n_features :]

        prob_phase0 = get_prob0(results_phase0[k], n_qubits)
        prob_phase1 = get_prob0(results_phase1[k], n_qubits)
        prob_phase2 = get_prob0(results_phase2[k], n_qubits)
        prob_phase3 = get_prob0(results_phase3[k], n_qubits)

        probs_phase0_sim_noisy[f"sample{i}"][f"f_{k}"] = prob_phase0
        probs_phase1_sim_noisy[f"sample{i}"][f"f_{k}"] = prob_phase1
        probs_phase2_sim_noisy[f"sample{i}"][f"f_{k}"] = prob_phase2
        probs_phase3_sim_noisy[f"sample{i}"][f"f_{k}"] = prob_phase3

        inner_product = np.exp(-1j * lambda_refs[f"sample{i}"] * times[k]) * (
            (prob_phase0 - prob_phase1) + 1j * (prob_phase2 - prob_phase3)
        )
        features.append(inner_product.real)
        if k != 0:
            features.append(inner_product.imag)
    data_sim_noisy.append([i, *features, all_expected_values[i]])

# Create column names for the DataFrame
columns = []
columns.append("sample_id")
for k in range(n_features):
    columns.append(f"f_{k} Re")
    if k != 0:
        columns.append(f"f_{k} Im")
columns.append("expected_value")

# Convert to a DataFrame
data_sim_noisy_df = pd.DataFrame(data_sim_noisy, columns=columns)
display(data_sim_noisy_df)

# Save the simulation data
data_sim_noisy_df.to_json(f"{path}/data_sim_noisy_df.json")
with open(f"{path}/probs_phase0_sim_noisy.json", "w") as f:
    json.dump(probs_phase0_sim_noisy, f)

with open(f"{path}/probs_phase1_sim_noisy.json", "w") as f:
    json.dump(probs_phase1_sim_noisy, f)

with open(f"{path}/probs_phase2_sim_noisy.json", "w") as f:
    json.dump(probs_phase2_sim_noisy, f)

with open(f"{path}/probs_phase3_sim_noisy.json", "w") as f:
    json.dump(probs_phase3_sim_noisy, f)

,sample_id,f_0 Re,f_1 Re,f_1 Im,f_2 Re,f_2 Im,f_3 Re,f_3 Im,f_4 Re,f_4 Im,...,f_5 Im,f_6 Re,f_6 Im,f_7 Re,f_7 Im,f_8 Re,f_8 Im,f_9 Re,f_9 Im,expected_value
0,0,0.980469,0.414519,-0.196669,-0.124413,-0.251420,-0.115187,-0.477212,0.261527,-0.341515,...,-0.059563,-0.495661,0.297585,-0.220554,0.049595,0.112642,-0.178164,0.368294,0.036212,2.653082
1,1,0.979492,0.856371,-0.223990,0.615062,-0.411922,0.342600,-0.528439,0.192524,-0.496673,...,-0.343620,0.100595,-0.162242,0.112704,-0.060585,0.166130,-0.092151,0.283934,-0.243153,0.865078
2,2,0.967773,0.230937,-0.304498,-0.039656,0.116682,-0.312337,-0.088457,-0.309474,0.133238,...,-0.415614,-0.143058,-0.596920,-0.112214,-0.056922,-0.138375,0.156043,-0.181719,0.620830,1.138950
3,3,0.972656,0.496837,-0.140253,0.080979,-0.291592,0.192619,-0.096377,0.350416,0.462852,...,0.530921,0.088786,-0.085938,0.381995,0.089599,0.339086,0.266363,0.019513,-0.017760,4.367027
4,4,0.976562,0.589439,0.476121,-0.113240,0.407916,-0.300273,0.038279,0.103804,-0.330826,...,-0.296849,0.787106,0.003784,0.350046,0.338345,-0.034333,0.340690,-0.196547,0.100080,4.566077
5,5,0.977539,0.389773,0.054507,-0.038356,0.334336,0.234748,0.227540,0.346831,-0.389973,...,-0.499162,0.008418,0.035193,0.300055,-0.346551,0.222863,-0.521291,0.151298,-0.057798,3.720380
6,6,0.978516,0.647034,-0.303691,0.186511,-0.507961,0.077373,-0.409271,0.136008,-0.047763,...,0.127396,-0.241049,-0.171899,-0.168334,-0.030514,-0.208153,0.285213,-0.238295,0.549531,1.711405
7,7,0.975586,0.587036,-0.319863,0.079819,-0.054795,0.010833,0.361816,0.179669,0.227554,...,-0.014193,-0.070391,-0.051900,0.163325,-0.103416,0.317175,-0.503705,0.023454,-0.690301,0.920606
8,8,0.989258,0.334213,-0.350994,0.046408,0.024349,-0.165207,-0.232130,-0.716139,-0.080792,...,0.168093,-0.014027,-0.344188,-0.108922,-0.066990,0.254931,0.121852,-0.039389,0.045744,1.084233
9,9,0.976562,0.592646,0.096750,0.023824,0.139924,-0.071649,0.157518,0.255265,0.314008,...,0.440585,0.054557,0.230475,-0.360719,-0.093596,-0.418284,-0.201552,-0.049808,0.018417,3.526609
